In [ ]:
# Importing functions and classes we'll use

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from warnings import catch_warnings
from warnings import filterwarnings
from tqdm import tqdm
import keras
import sys
import scipy.stats
import json
import numpy.fft
import time
from decimal import Decimal
import math
import seaborn as sns
%matplotlib inline

In [ ]:
# fix random seed for reproducibility
tf.random.set_seed(1234)

In [ ]:
df = pd.read_excel('/content/sample_data/new_dataset.xlsx')
node1_delay = df[['node1_delay']]

dataset = node1_delay.values

# MA Filter Implementation

In [ ]:
# ASAP
class Metrics(object):
    def __init__(self, values):
        self.set_values( values )

    def set_values(self, values):
        self.values = values
        self.r = self.k = None

    @property
    def kurtosis(self):
        if self.k is None:
            self.k = scipy.stats.kurtosis(self.values)
        return self.k

    @property
    def roughness(self):
        if self.r is None:
            self.r = np.std(np.diff(self.values))
        return self.r

class ACF(Metrics):
    CORR_THRESH = 0.2
    def __init__(self, values, max_lag=None):
        super(ACF, self).__init__(values)
        if max_lag is None:
            max_lag = len(values) / 5
        self.max_lag = int(max_lag)
        self.max_acf = 0.0

        # Calculate autocorrelation via FFT
        # Demean
        demeaned = values - np.mean(values)
        # Pad data to power of 2
        l = int(2.0 ** (int(math.log(len(demeaned),2.0)) + 1))
        padded = np.append(demeaned, ([0.0] * (l - len(demeaned))))
        # FFT and inverse FFT
        F_f = numpy.fft.fft( padded )
        R_t = numpy.fft.ifft( F_f * np.conjugate(F_f) )
        self.correlations = R_t[:int(max_lag)].real / R_t[0].real

        # Find autocorrelation peaks
        self.peaks = []
        if len(self.correlations) >1 :
            positive = self.correlations[1] > self.correlations[0]
            max = 1
            for i in range(2, len(self.correlations)):
                if not positive and self.correlations[i] > self.correlations[i-1]:
                    max = i
                    positive = not positive
                elif positive and self.correlations[i] > self.correlations[max]:
                    max = i
                elif positive and self.correlations[i] < self.correlations[i-1]:
                    if max > 1 and self.correlations[max] > self.CORR_THRESH:
                        self.peaks.append(max)
                        if self.correlations[max] > self.max_acf:
                            self.max_acf = self.correlations[max]
                    positive = not positive
        # If there is no autocorrelation peak within the MAX_WINDOW boundary,
        # try windows from the largest to the smallest
        if len(self.peaks) <= 1:
            self.peaks = range(2, len(self.correlations))

def moving_average(data, _range):
    ret = np.cumsum(data)
    ret[_range:] = ret[int(_range):] - ret[:-int(_range)]
    return ret[int(_range) - 1:] / _range

def SMA(data, _range, slide):
    ret = moving_average(data, int(_range))[::int(slide)]
    return list(ret)

def binary_search(head,tail,data,min_obj,orig_kurt,window_size):
    while head <= tail:
        w = int(round((head + tail) / 2.0))
        smoothed = SMA(data,w,1)
        metrics  = Metrics(smoothed)
        if metrics.kurtosis >= orig_kurt:
            if metrics.roughness < min_obj:
                window_size = w
                min_obj = metrics.roughness
            head = w + 1
        else:
            tail = w - 1
    return window_size

def smooth_ASAP(data, max_window=5, resolution=None):
    data = np.array(data)
    # Preaggregate according to resolution
    slide_size = 1
    window_size = 1
    if resolution and len(data) >= 2 * resolution:
        slide_size = len(data) / resolution
        data = SMA(data, slide_size, slide_size)
    acf         = ACF(data, max_lag=len(data) / max_window)
    peaks       = acf.peaks
    orig_kurt   = acf.kurtosis
    min_obj     = acf.roughness
    lb          = 1
    largest_feasible = -1
    tail = len(data) / max_window
    for i in range(len(peaks) - 1, -1, -1):
        w = peaks[i]

        if w < lb or w == 1:
            break
        elif math.sqrt(1 - acf.correlations[w]) * window_size > math.sqrt(1 - acf.correlations[window_size]) * w:
            continue

        smoothed = SMA(data, w, 1)
        metrics = Metrics(smoothed)
        if metrics.roughness < min_obj and metrics.kurtosis >= orig_kurt:
            min_obj = metrics.roughness
            window_size = w
            lb = round( max(w*math.sqrt( (acf.max_acf -1) / (acf.correlations[w]-1) ), lb) )
    if largest_feasible > 0:
        if largest_feasible < len(peaks) - 2:
            tail = peaks[largest_feasible + 1]
        lb = max(lb, peaks[largest_feasible] + 1)

    window_size = binary_search(lb, tail, data, min_obj, orig_kurt, window_size)
    return window_size, slide_size

### ARIMA Model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
import warnings

warnings.filterwarnings("ignore")

# multistep sarima forecast
def sarima_multistep_forecast(history, config, window_size, n_steps):
    order, sorder, trend = config
    new_hist = history[:]
    yhat = []
    total_training_elapsed_time = []
    # define model
    for i in range(n_steps):

      model = SARIMAX(new_hist[-window_size:], order=order, seasonal_order=sorder, trend=trend,
                    enforce_stationarity=False, enforce_invertibility=False)

      # Record the starting time to train ARIMA Model
      training_start_time = time.time()

      # fit model
      model_fit = model.fit(disp=False)

      # Record the ending time of Training
      training_end_time = time.time()
      training_elapsed_time = training_end_time - training_start_time
      total_training_elapsed_time.append(training_elapsed_time)

      # make multistep forecast
      # yhat = model_fit.forecast(steps=n_steps)
      prediction = model_fit.predict(start=len(history[-window_size:]), end=len(history[-window_size:]))
      yhat = np.append(yhat, prediction)
      new_hist = np.append(new_hist, prediction)
      new_hist = new_hist[1:]
    return yhat, np.sum(total_training_elapsed_time), np.mean(total_training_elapsed_time)

# root mean squared error or rmse
def measure_rmse(actual, predicted):
  return sqrt(mean_squared_error(actual, predicted))

In [ ]:
node1_delay = df[['node1_delay']]

node1_delay_dataset = node1_delay.values

window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)
denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]
print(len(train), len(test))

cfg = ((1, 0, 1), (0, 0, 0, 0), 'c')
window_length = 285

horizons = [2, 4, 6, 8, 10]

for horizon in horizons:
    print(f'================== horizon = {horizon} ==========================')
    predictions = []
    ys = []
    # seed history with training dataset
    history = []
    history.extend(train)

    sum_total_training_elapsed_time, mean_total_training_elapsed_time = [], []


    # Record the starting time to generate predictions
    predictions_start_time = time.time()

    # step over each time-step in the test set
    # for i = 0
    # fit model and make forecast for history
    yhat, sum_training_elapsed_time, mean_training_elapsed_time = sarima_multistep_forecast(np.array(history), cfg, window_length, horizon)
    sum_total_training_elapsed_time.append(sum_training_elapsed_time)
    mean_total_training_elapsed_time.append(mean_training_elapsed_time)
    # store forecast in list of predictions
    predictions.append(yhat)
    ys.append(test[:horizon])
    # add actual observation to history for the next loop
    history.extend(test[:horizon])
    for i in tqdm(range(horizon, len(test))):
        # fit model and make forecast for history
        yhat, sum_training_elapsed_time, mean_training_elapsed_time = sarima_multistep_forecast(np.array(history), cfg, window_length, horizon)
        # store forecast in list of predictions
        predictions.append(yhat)
        ys.append(test[i:i+horizon])
        sum_total_training_elapsed_time.append(sum_training_elapsed_time)
        mean_total_training_elapsed_time.append(mean_training_elapsed_time)
        # add actual observation to history for the next loop
        history.append(test[i])

    # Record the ending time of generating predictions
    predictions_end_time = time.time()
    predictions_elapsed_time = predictions_end_time - predictions_start_time

    # estimate prediction error
    ys_converted = [array.tolist() for array in ys if len(array) == horizon]
    predictions_converted = [array.tolist() for array in predictions]

    testRMSE = np.sqrt(mean_squared_error(ys_converted, predictions_converted[:len(ys_converted)]))
    testMAE = mean_absolute_error(ys_converted, predictions_converted[:len(ys_converted)])

    print('ARIMA Test RMSE : %.5f' % (testRMSE))
    print('ARIMA Test MAE : %.5f' % (testMAE))
    print("ARIMA Total Time to train models : %.5f" % (np.sum(sum_total_training_elapsed_time)), "seconds")
    print("ARIMA mean Time to train models : %.5f" % (np.mean(mean_total_training_elapsed_time)), "seconds")
    print("ARIMA Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000
================== horizon = 2 ==========================


100%|██████████| 1998/1998 [11:27<00:00,  2.91it/s]


ARIMA Test RMSE : 0.02629
ARIMA Test MAE : 0.02119
ARIMA Total Time to train models : 653.75044 seconds
ARIMA mean Time to train models : 0.16352 seconds
ARIMA Elapsed Time To generate Predictions : 688.39788 seconds
================== horizon = 4 ==========================


100%|██████████| 1996/1996 [21:35<00:00,  1.54it/s]


ARIMA Test RMSE : 0.03295
ARIMA Test MAE : 0.02622
ARIMA Total Time to train models : 1244.10167 seconds
ARIMA mean Time to train models : 0.15575 seconds
ARIMA Elapsed Time To generate Predictions : 1296.05685 seconds
================== horizon = 6 ==========================


...

ARIMA Test RMSE : 0.03770
ARIMA Test MAE : 0.02971
ARIMA Total Time to train models : 1852.95325 seconds
ARIMA mean Time to train models : 0.15480 seconds
ARIMA Elapsed Time To generate Predictions : 1924.17167 seconds
================== horizon = 8 ==========================


100%|██████████| 1992/1992 [42:45<00:00,  1.29s/it]


ARIMA Test RMSE : 0.04142
ARIMA Test MAE : 0.03255
ARIMA Total Time to train models : 2474.79536 seconds
ARIMA mean Time to train models : 0.15522 seconds
ARIMA Elapsed Time To generate Predictions : 2566.81526 seconds
================== horizon = 10 ==========================


100%|██████████| 1990/1990 [53:25<00:00,  1.61s/it]

ARIMA Test RMSE : 0.04443
ARIMA Test MAE : 0.03492
ARIMA Total Time to train models : 3095.37520 seconds
ARIMA mean Time to train models : 0.15547 seconds
ARIMA Elapsed Time To generate Predictions : 3206.97499 seconds
